In [6]:
from hnswlib import Index
import numpy as np
from sentence_transformers import SentenceTransformer
import time
import pickle
import hnswlib
import torch

/Users/artashesmezhlumyan/Desktop/capstone/capstonevenv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/artashesmezhlumyan/Desktop/capstone/capstonevenv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')

/Users/artashesmezhlumyan/Desktop/capstone/capstonevenv/lib/python3.9/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


## Cosie search

In [8]:

def search_similar_sentences(query, index, doc_list, k=5):
    query_embedding = model.encode([query])

    labels, distances = index.knn_query(query_embedding, k=k)
    similar_sentences_with_scores = [(doc_list[label], 1 - distance) for label, distance in zip(labels[0], distances[0])]

    return similar_sentences_with_scores


with open("embeddings.pkl", "rb") as fIn:
    stored_data = pickle.load(fIn)
    stored_sentences = stored_data["sentences"]
    stored_embeddings = stored_data["embeddings"]

# Ensure stored_embeddings is on CPU and converted to NumPy
if isinstance(stored_embeddings, torch.Tensor):
    stored_embeddings = stored_embeddings.cpu().numpy()  # Convert to CPU and NumPy array
print(stored_embeddings.shape)
dimension = stored_embeddings.shape[1]
print(dimension)
p = hnswlib.Index(space='cosine', dim=dimension)
p.init_index(max_elements=10000, ef_construction=200, M=16)
print(type(stored_embeddings))
print(stored_embeddings.shape)
p.add_items(stored_embeddings)  # Now stored_embeddings is a NumPy array on CPU
p.set_ef(50)  # Setting ef, which controls the recall

new_sentence = "My sister's leg broke"
top_similar_sentences = search_similar_sentences(new_sentence, p, stored_sentences, k=3)

print("Top 3 similar sentences are:")
for i, sentence in enumerate(top_similar_sentences):
    print(f"{i+1}. {sentence}")


(3, 384)
384
<class 'numpy.ndarray'>
(3, 384)
Top 3 similar sentences are:
1. ('  We describe a new algorithm, the $(k,\\ell)$-pebble game with colors, and use\nit obtain a characterization of the family of $(k,\\ell)$-sparse graphs and\nalgorithmic solutions to a family of problems concerning tree decompositions of\ngraphs. Special instances of sparse graphs appear in rigidity theory and have\nreceived increased attention in recent years. In particular, our colored\npebbles generalize and strengthen the previous results of Lee and Streinu and\ngive a new proof of the Tutte-Nash-Williams characterization of arboricity. We\nalso present a new decomposition that certifies sparsity based on the\n$(k,\\ell)$-pebble game with colors. Our work also exposes connections between\npebble game algorithms and previous sparse graph algorithms by Gabow, Gabow and\nWestermann and Hendrickson.\n', 0.028408825397491455)
2. ("  The evolution of Earth-Moon system is described by the dark matter field\nfl

## IP

In [9]:

def search_similar_sentences(query, index, doc_list, k=5):
    query_embedding = model.encode([query])

    labels, distances = index.knn_query(query_embedding, k=k)
    similar_sentences_with_scores = [(doc_list[label], 1 - distance) for label, distance in zip(labels[0], distances[0])]

    return similar_sentences_with_scores


with open("embeddings.pkl", "rb") as fIn:
    stored_data = pickle.load(fIn)
    stored_sentences = stored_data["sentences"]
    stored_embeddings = stored_data["embeddings"]

# Ensure stored_embeddings is on CPU and converted to NumPy
if isinstance(stored_embeddings, torch.Tensor):
    stored_embeddings = stored_embeddings.cpu().numpy()  # Convert to CPU and NumPy array

dimension = stored_embeddings.shape[1]
p = hnswlib.Index(space='ip', dim=dimension)
p.init_index(max_elements=10000, ef_construction=200, M=16)
p.add_items(stored_embeddings)  # Now stored_embeddings is a NumPy array on CPU
p.set_ef(50)  # Setting ef, which controls the recall

new_sentence = "My sister's leg broke"
top_similar_sentences = search_similar_sentences(new_sentence, p, stored_sentences, k=3)

print("Top 3 similar sentences are:")
for i, sentence in enumerate(top_similar_sentences):
    print(f"{i+1}. {sentence}")


Top 3 similar sentences are:
1. ('  We describe a new algorithm, the $(k,\\ell)$-pebble game with colors, and use\nit obtain a characterization of the family of $(k,\\ell)$-sparse graphs and\nalgorithmic solutions to a family of problems concerning tree decompositions of\ngraphs. Special instances of sparse graphs appear in rigidity theory and have\nreceived increased attention in recent years. In particular, our colored\npebbles generalize and strengthen the previous results of Lee and Streinu and\ngive a new proof of the Tutte-Nash-Williams characterization of arboricity. We\nalso present a new decomposition that certifies sparsity based on the\n$(k,\\ell)$-pebble game with colors. Our work also exposes connections between\npebble game algorithms and previous sparse graph algorithms by Gabow, Gabow and\nWestermann and Hendrickson.\n', 0.026975154876708984)
2. ("  The evolution of Earth-Moon system is described by the dark matter field\nfluid model proposed in the Meeting of Division 

## L2

In [10]:

def search_similar_sentences(query, index, doc_list, k=5):
    query_embedding = model.encode([query])

    labels, distances = index.knn_query(query_embedding, k=k)
    similar_sentences_with_scores = [(doc_list[label], 1 - distance) for label, distance in zip(labels[0], distances[0])]

    return similar_sentences_with_scores


with open("embeddings.pkl", "rb") as fIn:
    stored_data = pickle.load(fIn)
    stored_sentences = stored_data["sentences"]
    stored_embeddings = stored_data["embeddings"]

# Ensure stored_embeddings is on CPU and converted to NumPy
if isinstance(stored_embeddings, torch.Tensor):
    stored_embeddings = stored_embeddings.cpu().numpy()  # Convert to CPU and NumPy array

dimension = stored_embeddings.shape[1]
p = hnswlib.Index(space='l2', dim=dimension)
p.init_index(max_elements=10000, ef_construction=200, M=16)
p.add_items(stored_embeddings)  # Now stored_embeddings is a NumPy array on CPU
p.set_ef(50)  # Setting ef, which controls the recall

new_sentence = "My sister's leg broke"
top_similar_sentences = search_similar_sentences(new_sentence, p, stored_sentences, k=3)

print("Top 3 similar sentences are:")
for i, sentence in enumerate(top_similar_sentences):
    print(f"{i+1}. {sentence}")


Top 3 similar sentences are:
1. ('  A fully differential calculation in perturbative quantum chromodynamics is\npresented for the production of massive photon pairs at hadron colliders. All\nnext-to-leading order perturbative contributions from quark-antiquark,\ngluon-(anti)quark, and gluon-gluon subprocesses are included, as well as\nall-orders resummation of initial-state gluon radiation valid at\nnext-to-next-to-leading logarithmic accuracy. The region of phase space is\nspecified in which the calculation is most reliable. Good agreement is\ndemonstrated with data from the Fermilab Tevatron, and predictions are made for\nmore detailed tests with CDF and DO data. Predictions are shown for\ndistributions of diphoton pairs produced at the energy of the Large Hadron\nCollider (LHC). Distributions of the diphoton pairs from the decay of a Higgs\nboson are contrasted with those produced from QCD processes at the LHC, showing\nthat enhanced sensitivity to the signal can be obtained with ju